In [1]:
import datetime
from statistics import mean
from pprint import pprint

import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

from data import kfolds, train_test_split, classification_dataset
from train import best_num_epochs, train_model
from layers import SeluConv3D, SeluDense
from plot import plot_slice, plot_volume_animation
from config import (
    LIDC_SMALL_NEG_TFRECORD,
    LIDC_SMALL_POS_TFRECORD,
    LIDC_BIG_NEG_TFRECORD,
    LIDC_BIG_POS_TFRECORD,
    SMALL_PATCH_SHAPE,
    BIG_PATCH_SHAPE,
)

%matplotlib inline
plt.rcParams["figure.figsize"] = [15, 7]

In [2]:
# Hyperparameters
val_perc = 0.2
k = 10
patience = 30
extra_epochs = 5
batch_size = 16
learning_rate = 1e-5
dropout_rate = 0.6

In [3]:
dataset, total_samples = classification_dataset(
    LIDC_SMALL_NEG_TFRECORD,
    LIDC_BIG_NEG_TFRECORD,
    LIDC_SMALL_POS_TFRECORD,
    LIDC_BIG_POS_TFRECORD,
    return_size=True,
)
total_samples

754

In [4]:
def build_model():
    input_small = keras.Input(SMALL_PATCH_SHAPE, name="input_small")
    x_small = SeluConv3D(
        filters=32,
        kernel_size=3,
        name="small_selu_conv3d_1",
    )(input_small)
    x_small = keras.layers.MaxPooling3D((1, 2, 2), name="small_maxpool_1")(x_small)
    x_small = SeluConv3D(
        filters=64,
        kernel_size=3,
        name="small_selu_conv3d_2",
    )(x_small)
    x_small = keras.layers.MaxPooling3D((1, 2, 2), name="small_maxpool_2")(x_small)
    x_small = SeluConv3D(
        filters=128,
        kernel_size=3,
        name="small_selu_conv3d_3",
    )(x_small)
    x_small = keras.layers.MaxPooling3D((1, 2, 2), name="small_maxpool_3")(x_small)
    x_small = SeluConv3D(
        filters=256,
        kernel_size=3,
        name="small_selu_conv3d_4",
    )(x_small)
    x_small = keras.layers.Flatten(name="flatten_small")(x_small)

    input_big = keras.Input(BIG_PATCH_SHAPE, name="input_big")
    x_big = keras.layers.MaxPooling3D((2, 2, 2), name="big_maxpool_0")(input_big)
    x_big = SeluConv3D(
        filters=32,
        kernel_size=3,
        name="big_selu_conv3d_1",
    )(x_big)
    x_big = keras.layers.MaxPooling3D((1, 2, 2), name="big_maxpool_1")(x_big)
    x_big = SeluConv3D(
        filters=64,
        kernel_size=3,
        name="big_selu_conv3d_2",
    )(x_big)
    x_big = keras.layers.MaxPooling3D((1, 2, 2), name="big_maxpool_2")(x_big)
    x_big = SeluConv3D(
        filters=128,
        kernel_size=3,
        name="big_selu_conv3d_3",
    )(x_big)
    x_big = keras.layers.MaxPooling3D((1, 2, 2), name="big_maxpool_3")(x_big)
    x_big = SeluConv3D(
        filters=256,
        kernel_size=3,
        name="big_selu_conv3d_4",
    )(x_big)
    x_big = keras.layers.Flatten(name="flatten_big")(x_big)

    x = keras.layers.concatenate([x_small, x_big], name="concatenate")
    x = SeluDense(128, name="selu_dense")(x)
    x = keras.layers.AlphaDropout(dropout_rate, name="alpha_dropout")(x)
    x = keras.layers.Dense(1, activation="sigmoid", name="final_dense")(x)

    cnn_3d = keras.Model(inputs=[input_small, input_big], outputs=x, name="3dcnn")

    return cnn_3d

In [5]:
def compile_model(model):
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate),
        loss="binary_crossentropy",
        metrics=[
            keras.metrics.AUC(name="auc"),
            keras.metrics.BinaryAccuracy(name="accuracy"),
        ],
    )

In [6]:
train_dataset, val_dataset = train_test_split(
    dataset, test_perc=val_perc, cardinality=total_samples
)
val_dataset = val_dataset.batch(1)
train_dataset = (
    train_dataset.batch(batch_size)
    .cache()  # must be called before shuffle
    .shuffle(buffer_size=64, reshuffle_each_iteration=True)
    .prefetch(tf.data.experimental.AUTOTUNE)
)
cnn = build_model()
compile_model(cnn)
log_dir = f"logs/baseline-lidc"
num_epochs = best_num_epochs(
    cnn,
    train_dataset,
    val_dataset,
    patience,
    "val_accuracy",
    log_dir,
    verbose_training=1,
)
f"{num_epochs = }"

Epoch 1/1000
38/38 [==============================] - 3s 74ms/step - loss: 0.7579 - auc: 0.6006 - accuracy: 0.5629 - val_loss: 0.6100 - val_auc: 0.7798 - val_accuracy: 0.7467
Epoch 2/1000
38/38 [==============================] - 2s 60ms/step - loss: 0.6481 - auc: 0.6986 - accuracy: 0.6490 - val_loss: 0.5377 - val_auc: 0.8365 - val_accuracy: 0.8133
Epoch 3/1000
38/38 [==============================] - 2s 61ms/step - loss: 0.5849 - auc: 0.7592 - accuracy: 0.6821 - val_loss: 0.5799 - val_auc: 0.8468 - val_accuracy: 0.7867
Epoch 4/1000
38/38 [==============================] - 2s 60ms/step - loss: 0.5602 - auc: 0.7905 - accuracy: 0.7202 - val_loss: 0.5640 - val_auc: 0.8721 - val_accuracy: 0.8067
Epoch 5/1000
38/38 [==============================] - 2s 59ms/step - loss: 0.5373 - auc: 0.8059 - accuracy: 0.7152 - val_loss: 0.6161 - val_auc: 0.8647 - val_accuracy: 0.7933
Epoch 6/1000
38/38 [==============================] - 2s 62ms/step - loss: 0.5114 - auc: 0.8220 - accuracy: 0.7384 - val_loss

'num_epochs = 53'

In [7]:
metrics = {"auc": [], "accuracy": []}
sess_id = 0
for train_dataset, test_dataset in tqdm(
    kfolds(k, dataset, cardinality=total_samples), total=k
):
    test_dataset = test_dataset.batch(1)
    train_dataset = (
        train_dataset.batch(batch_size)
        .cache()  # must be called before shuffle
        .shuffle(buffer_size=64, reshuffle_each_iteration=True)
        .prefetch(tf.data.experimental.AUTOTUNE)
    )
    cnn = build_model()
    compile_model(cnn)
    model_fname = f"models/baseline-lidc-{sess_id}.h5"
    cnn = train_model(
        cnn,
        train_dataset,
        num_epochs + extra_epochs,
        model_fname,
    )
    test_metrics = cnn.evaluate(test_dataset, return_dict=True, verbose=0)
    for metric_name, metric_value in test_metrics.items():
        if metric_name in metrics:
            metrics[metric_name].append(metric_value)
    print(f"Last accuracy: {metrics['accuracy'][-1]}")
    sess_id += 1
mean_metrics = {
    metric_name: mean(metric_values)
    for metric_name, metric_values in metrics.items()
    if metric_name in metrics
}
print("==============================================")
pprint(mean_metrics)

Last accuracy: 0.8266666531562805
Last accuracy: 0.7733333110809326
Last accuracy: 0.7333333492279053
Last accuracy: 0.7866666913032532
Last accuracy: 0.7066666483879089
Last accuracy: 0.6933333277702332
Last accuracy: 0.8399999737739563
Last accuracy: 0.6666666865348816
Last accuracy: 0.7866666913032532
Last accuracy: 0.746666669845581

{'accuracy': 0.7560000002384186, 'auc': 0.8256904542446136}


In [ ]:
patches, label = next(iter(test_dataset.skip(6)))
print(f"label: {label[0][0].numpy()}")
prediction = cnn(patches, training=False)
print(f"prediction: {prediction[0][0].numpy()}")
plot_volume_animation(patches[0][0, :])